In [1]:
import warnings
warnings.filterwarnings('ignore',category=RuntimeWarning)
import xarray as xr
import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt
import seaborn as sns
import glob,os,sys
from tqdm import tqdm
import datetime
import proplot as plot
from netCDF4 import Dataset
from wrf import getvar
import json,pickle
from scipy.ndimage import uniform_filter1d
from scipy.ndimage import gaussian_filter

sys.path.insert(1, '/work/08350/tg876493/stampede2/python_codes/2020_TC_CRF/dev/freddy0218/tools/')
import derive_var
import read_and_proc,preproc_noensemble
%matplotlib inline
plot.rc.update({'figure.facecolor':'w','axes.labelweight':'ultralight',
                'tick.labelweight':'ultralight','gridminor.linestyle':'--','title.weight':'normal','linewidth':0.5})

In [2]:
#Function####################################################
def nearest_index(array, value):
    idx = (np.abs(array-value)).argmin()
    return idx.values
def forward_diff(arrayin=None,delta=None,axis=None):
    result = []
    if axis==0:
        for i in range(0,arrayin.shape[axis]-1):
            temp = (arrayin[i+1,:]-arrayin[i,:])/(delta)
            result.append(temp)
        return np.asarray(result)
    
def do_PCA(var=None,timezoom=None,smooth='Yes',gaussian=0.9,fromcenter='Yes',inradius=None,outradius=None,donormal='Yes',do_PCA='Yes',do_center='Yes'):
    """
    Var: Input variable (must have 4 dimensions! Time-pres-theta-radius)
    """
    if smooth=='Yes':
        from scipy.ndimage import gaussian_filter
        normal_var = []
        if donormal=='Yes':
            for presindex in range(len(var[0,:,0,0])):
                normal_var.append(gaussian_filter(normalize_inner(var[:,presindex,:,:],outradius,'Yes'),sigma=gaussian))
        elif donormal=='No':
            if do_center=='Yes':
                for presindex in range(len(var[0,:,0,0])):
                    normal_var.append(gaussian_filter(normalize_inner(var[:,presindex,:,:],outradius,'No','Yes'),sigma=gaussian))
            elif do_center=='No':
                for presindex in range(len(var[0,:,0,0])):
                    normal_var.append(gaussian_filter(normalize_inner(var[:,presindex,:,:],outradius,'No','No'),sigma=gaussian))  
        normal_var = np.swapaxes(np.asarray(normal_var),0,1)
    else:
        normal_var = []
        for presindex in range(len(var[0,:,0,0])):
            normal_var.append(normalize_inner(var[:,presindex,:,:],outradius))
        normal_var = np.swapaxes(np.asarray(normal_var),0,1)
    if fromcenter=='Yes':
        normal_varf = np.asarray([normal_var[i,:,:,:outradius].flatten() \
                                  for i in range(len(normal_var[timezoom[0]:timezoom[1],0,0,0]))])
    elif fromcenter=='No':
        normal_varf = np.asarray([normal_var[i,:,:,inradius:outradius].flatten() \
                                  for i in range(len(normal_var[timezoom[0]:timezoom[1],0,0,0]))])        
    #print("--Finish preprocesing--")
    if do_PCA=='Yes':
        from sklearn.decomposition import PCA
        import time
        start_time = time.time()
        skpcaVAR = PCA()
        skpcaVAR.fit(normal_varf.copy())
        #print("--- %s seconds ---" % (time.time() - start_time))
        return skpcaVAR,normal_var,normal_varf
    else:
        return normal_var,normal_varf

def normalize_inner(var=None,outerradius=None,standard='Yes',docenter='No'):
    PWper_ctrl = []
    for indx in range(len(var[:,0,0])):
        if docenter=='Yes':
            temp = var[indx,:,:outerradius]-np.nanmean(var[indx,:,:outerradius],axis=(0,1))
        elif docenter=='No':
            temp = var[indx,:,:outerradius]
        if standard=='Yes':
            PWper_ctrl.append((temp-np.nanmean(temp))/np.nanstd(temp))
        elif standard=='No':
            PWper_ctrl.append((temp))
    del temp
    return np.asarray(PWper_ctrl)

### Velocities

In [3]:
import gc
def do_uradvtan_nonensemble(originpath=None,coor_pres=None,expname=None,savepath=None):
    """
    Derive urad/vtan and save to /scratch
    """
    ctrlvar_dict = read_and_proc.read_some_azimuth_fields(fileloc=[glob.glob(originpath+str(expname)+'/azim_U_*')[0],\
                                                     glob.glob(originpath+str(expname)+'/azim_V_*')[0]],fieldname=['U','V'])
    r500 = nearest_index(ctrlvar_dict['U']['U'].radius,500)
    ctrlhUrad,ctrlhVtan,_ = derive_var.ruppert_vtmax_calc(ctrlvar_dict['U']['U'].data,\
                                                          ctrlvar_dict['V']['V'].data,r500,ctrlvar_dict['U']['U'][:,0,:,:].azmiuth.values,None)
    read_and_proc.save_to_pickle(savepath+'/'+str(expname)+'/'+'urad',ctrlhUrad,'PICKLE')
    read_and_proc.save_to_pickle(savepath+'/'+str(expname)+'/'+'vtan',ctrlhVtan,'PICKLE')
        
    del ctrlvar_dict,ctrlhUrad,ctrlhVtan
    gc.collect()
    return None  

def preproc_dtuvwqv(uvtpath=None,originpath=None,expname=None):
    ###############################################################################################
    # urad,vtan,theta & swapaxes
    ###############################################################################################
    urad = np.swapaxes(np.asarray(read_and_proc.depickle(uvtpath+str(expname)+'/urad')),0,1)
    vtan = np.swapaxes(np.asarray(read_and_proc.depickle(uvtpath+str(expname)+'/vtan')),0,1)
    theta = np.swapaxes(np.asarray(read_and_proc.depickle(uvtpath+str(expname)+'/theta')),0,1)
    ###############################################################################################
    # w,qv 
    ###############################################################################################
    var_dict = read_and_proc.read_some_azimuth_fields(fileloc=[glob.glob(originpath+str(expname)+'/azim_W*')[0],\
                                                                   glob.glob(originpath+str(expname)+'/azim_QVAPOR_*')[0]],fieldname=['W','QVAPOR'])
    #ctrl_t2,ctrl_t4 = nearest_index(ctrlvar_dict['W']['W'].time/24,0.5)-1,nearest_index(ctrlvar_dict['W']['W'].time/24,7)-1
    r500=nearest_index(var_dict['W']['W'].radius,500)
    ###############################################################################################
    # dtheta
    ###############################################################################################
    if expname=='ctl':
        dtheta = forward_diff(gaussian_filter(theta,sigma=[3,0,0,0]),60*60,0)
    elif (expname=='ncrf_36h') or (expname=='lwcrf'):
        ctrl_thetaA = np.swapaxes(np.asarray(read_and_proc.depickle(uvtpath+'/ctl/theta')),0,1)
        thetaL = read_and_proc.add_ctrl_before_senstart(ctrl_thetaA,theta,'NCRF36','Yes')
        dtheta = forward_diff(gaussian_filter(thetaL,sigma=[3,0,0,0]),60*60,0)
        del ctrl_thetaA,thetaL
        gc.collect()
        ctrl_uradA = np.swapaxes(np.asarray(read_and_proc.depickle(uvtpath+'/ctl/urad')),0,1)
        ctrl_vtanA = np.swapaxes(np.asarray(read_and_proc.depickle(uvtpath+'/ctl/vtan')),0,1)
        uradL = read_and_proc.add_ctrl_before_senstart(ctrl_uradA,urad,'NCRF36','Yes')
        vtanL = read_and_proc.add_ctrl_before_senstart(ctrl_vtanA,vtan,'NCRF36','Yes')
        del ctrl_uradA,ctrl_vtanA
        gc.collect()
        ctrlvar_dict = read_and_proc.read_some_azimuth_fields(fileloc=[glob.glob(originpath+'ctl/azim_W*')[0],\
                                                                       glob.glob(originpath+'ctl/azim_QVAPOR_*')[0]],fieldname=['W','QVAPOR'])
        wL = read_and_proc.add_ctrl_before_senstart(ctrlvar_dict['W']['W'],var_dict['W']['W'],'NCRF36','Yes')
        qvL = read_and_proc.add_ctrl_before_senstart(ctrlvar_dict['QVAPOR']['QVAPOR'],var_dict['QVAPOR']['QVAPOR'],'NCRF36','Yes')
        del ctrlvar_dict
        gc.collect()
    elif (expname=='ncrf_60h'):
        ctrl_thetaA = np.swapaxes(np.asarray(read_and_proc.depickle(uvtpath+'/ctl/theta')),0,1)
        thetaL = read_and_proc.add_ctrl_before_senstart(ctrl_thetaA,theta,'NCRF60','Yes')
        dtheta = forward_diff(gaussian_filter(thetaL,sigma=[3,0,0,0]),60*60,0)
        del ctrl_thetaA,thetaL
        gc.collect()
        ctrl_uradA = np.swapaxes(np.asarray(read_and_proc.depickle(uvtpath+'/ctl/urad')),0,1)
        ctrl_vtanA = np.swapaxes(np.asarray(read_and_proc.depickle(uvtpath+'/ctl/vtan')),0,1)
        uradL = read_and_proc.add_ctrl_before_senstart(ctrl_uradA,urad,'NCRF60','Yes')
        vtanL = read_and_proc.add_ctrl_before_senstart(ctrl_vtanA,vtan,'NCRF60','Yes')
        del ctrl_uradA,ctrl_vtanA
        gc.collect()
        ctrlvar_dict = read_and_proc.read_some_azimuth_fields(fileloc=[glob.glob(originpath+'ctl/azim_W*')[0],\
                                                                       glob.glob(originpath+'ctl/azim_QVAPOR_*')[0]],fieldname=['W','QVAPOR'])
        wL = read_and_proc.add_ctrl_before_senstart(ctrlvar_dict['W']['W'],var_dict['W']['W'],'NCRF60','Yes')
        qvL = read_and_proc.add_ctrl_before_senstart(ctrlvar_dict['QVAPOR']['QVAPOR'],var_dict['QVAPOR']['QVAPOR'],'NCRF60','Yes')
        del ctrlvar_dict
        gc.collect()
    del theta
    gc.collect()
    ###############################################################################################
    # smooth
    ###############################################################################################
    if expname=='ctl':
        urads,vtans = gaussian_filter(urad,sigma=[3,0,0,0]),gaussian_filter(vtan,sigma=[3,0,0,0])
        ws,qvs = gaussian_filter(var_dict['W']['W'],sigma=[3,0,0,0]),gaussian_filter(var_dict['QVAPOR']['QVAPOR'],sigma=[3,0,0,0])
        del urad,vtan,var_dict
    else:
        urads,vtans = gaussian_filter(uradL,sigma=[3,0,0,0]),gaussian_filter(vtanL,sigma=[3,0,0,0])
        ws,qvs = gaussian_filter(wL,sigma=[3,0,0,0]),gaussian_filter(qvL,sigma=[3,0,0,0])            
        del uradL,vtanL,wL,qvL,var_dict    
    gc.collect()
            #print('memb_0'+str(i))
            
    urads = np.nan_to_num(urads)
    vtans = np.nan_to_num(vtans)
    ws = np.nan_to_num(ws)
    qvs = np.nan_to_num(qvs)
    dtheta = np.nan_to_num(dtheta)
    print("---Finish!---")
    return dtheta,urads,vtans,ws,qvs,r500

#### Original PCA

In [4]:
#path_coor = '/scratch/06040/tg853394/tc/output/redux/maria/ctl/post/U.nc'
#coor = xr.open_dataset(path_coor)
#######################################################################################################################
# Radial/tangential winds
#######################################################################################################################
#from tqdm import tqdm
#exp_name=['ctl','ncrf_36h','ncrf_60h','lwcrf']
#for inx,name in tqdm(enumerate(exp_name)):
#    do_uradvtan_nonensemble(originpath='/scratch/06040/tg853394/tc/output/redux/maria/',coor_pres=coor.pres,\
#                expname=name,savepath='/scratch/08350/tg876493/ruppert2020/')
#######################################################################################################################
# Preproc data
#######################################################################################################################
ctl_dtheta,ctl_urad,ctl_vtan,ctl_w,ctl_qv,r500 = preproc_dtuvwqv(uvtpath='/scratch/08350/tg876493/ruppert2020/',
                                                                originpath='/scratch/06040/tg853394/tc/output/redux/maria/',expname='ctl')
ncrf36h_dtheta,ncrf36h_urad,ncrf36h_vtan,ncrf36h_w,ncrf36h_qv,_ = preproc_dtuvwqv(uvtpath='/scratch/08350/tg876493/ruppert2020/',
                                                                originpath='/scratch/06040/tg853394/tc/output/redux/maria/',expname='ncrf_36h')
ncrf60h_dtheta,ncrf60h_urad,ncrf60h_vtan,ncrf60h_w,ncrf60h_qv,_ = preproc_dtuvwqv(uvtpath='/scratch/08350/tg876493/ruppert2020/',
                                                                originpath='/scratch/06040/tg853394/tc/output/redux/maria/',expname='ncrf_60h')
lwcrf_dtheta,lwcrf_urad,lwcrf_vtan,lwcrf_w,lwcrf_qv,_ = preproc_dtuvwqv(uvtpath='/scratch/08350/tg876493/ruppert2020/',
                                                                originpath='/scratch/06040/tg853394/tc/output/redux/maria/',expname='lwcrf')

---Finish!---
---Finish!---
---Finish!---
---Finish!---


In [5]:
origin_path = '/scratch/06040/tg853394/tc/output/redux/maria/'
ctrlvar_dict = read_and_proc.read_some_azimuth_fields(fileloc=[origin_path+'ctl/azim_U_0-144hr.nc'],fieldname=['U'])
ncrf36var_dict = read_and_proc.read_some_azimuth_fields(fileloc=[origin_path+'ncrf_36h/azim_U_36-144hr.nc'],fieldname=['U'])
ncrf60var_dict = read_and_proc.read_some_azimuth_fields(fileloc=[origin_path+'ncrf_60h/azim_U_60-144hr.nc'],fieldname=['U'])

ctrl_t2,ctrl_t4 = nearest_index(ctrlvar_dict['U']['U'].time/24,1),nearest_index(ctrlvar_dict['U']['U'].time/24,5.5)
ncrf36_t2,ncrf36_t4 = nearest_index(ncrf36var_dict['U']['U'].time/24,1),nearest_index(ncrf36var_dict['U']['U'].time/24,5.5)
ncrf60_t2,ncrf60_t4 = nearest_index(ncrf60var_dict['U']['U'].time/24,1),nearest_index(ncrf60var_dict['U']['U'].time/24,5.5)
#del ctrlvar_dict,ncrf36var_dict,ncrf60var_dict
#gc.collect()

In [6]:
index_arrays = [np.ones((x.shape), dtype=int)*i for i, x in enumerate([ctl_dtheta[24:120],ncrf36h_dtheta[36:120],ncrf60h_dtheta[60:120],lwcrf_dtheta[36:120]])]
alltheta = np.concatenate(([y for y in [ctl_dtheta[24:120],ncrf36h_dtheta[36:120],ncrf60h_dtheta[60:120],lwcrf_dtheta[36:120]]]))
del ctl_dtheta,ncrf36h_dtheta,ncrf60h_dtheta,lwcrf_dtheta
gc.collect()
allurad = np.concatenate(([y for y in [ctl_urad[:-1][24:120],ncrf36h_urad[:-1][36:120],ncrf60h_urad[:-1][60:120],lwcrf_urad[:-1][36:120]]]))
del ctl_urad,ncrf36h_urad,ncrf60h_urad,lwcrf_urad
gc.collect()
allvtan = np.concatenate(([y for y in [ctl_vtan[:-1][24:120],ncrf36h_vtan[:-1][36:120],ncrf60h_vtan[:-1][60:120],lwcrf_vtan[:-1][36:120]]]))
del ctl_vtan,ncrf36h_vtan,ncrf60h_vtan,lwcrf_vtan
gc.collect()
allw = np.concatenate(([y for y in [ctl_w[:-1][24:120],ncrf36h_w[:-1][36:120],ncrf60h_w[:-1][60:120],lwcrf_w[:-1][36:120]]]))
del ctl_w,ncrf36h_w,ncrf60h_w,lwcrf_w
gc.collect()
allqv = np.concatenate(([y for y in [ctl_qv[:-1][24:120],ncrf36h_qv[:-1][36:120],ncrf60h_qv[:-1][60:120],lwcrf_qv[:-1][36:120]]]))
del ctl_qv,ncrf36h_qv,ncrf60h_qv,lwcrf_qv
gc.collect()

0

##### PCA

In [7]:
def save_PCA(var=None,varname=['U','V','W','QV','dthdt'],savepath=None):
    ###################################################################
    # Do PCA
    #CTRL_PCAdict,CTRL_flatvardict,CTRL_origvardict = {},{},{}
    for indx,obj in tqdm(enumerate(var)):
        temp1,temp3,temp2 = do_PCA(var=obj,timezoom=[0,-1],smooth='Yes',
                                   gaussian=[0,0,0],fromcenter='Yes',inradius=None,outradius=r500,donormal='No',do_PCA='Yes',do_center='No')
        read_and_proc.save_to_pickle(savepath+'pca/'+str(varname[indx])+'_pca',temp1)
        read_and_proc.save_to_pickle(savepath+'origvar/'+str(varname[indx])+'_origvar',temp3)
        read_and_proc.save_to_pickle(savepath+'flatvar/'+str(varname[indx])+'_flatvar',temp2) 
        del temp1,temp2,temp3
        gc.collect()        
    return None

In [8]:
save_PCA(var=[alltheta],varname=['dthdt'],savepath='/scratch/08350/tg876493/ruppert2020/output/')
read_and_proc.save_to_pickle('/scratch/08350/tg876493/ruppert2020/output/index_array',index_arrays) 
#var=[allurad,allvtan,allw,allqv,alltheta],varname=['U','V','W','QV','dthdt']

1it [15:20, 920.59s/it]


In [5]:
ctrl_ugrad_b = forward_diff(gaussian_filter(np.swapaxes(np.asarray(ctrlhUrad),0,1),sigma=[3,0,0,0]),60*60,0)
ctrl_vgrad_b = forward_diff(gaussian_filter(np.swapaxes(np.asarray(ctrlhVtan),0,1),sigma=[3,0,0,0]),60*60,0)
ctrl_wgrad_b = forward_diff(gaussian_filter(ctrlvar_dict['W']['W'].data,sigma=[3,0,0,0]),60*60,0)
del ctrlhUrad,ctrlhVtan,ncrf_36hUrad,ncrf_36hVtan,ncrf_60hUrad,ncrf_60hVtan,ctrlvar_dict['W']['W'],ncrf60var_dict,ncrf36var_dict
gc.collect()

40

#### Radiative

In [4]:
def preproc_onevar(varname=None,originpath=None,expname=None):
    ###############################################################################################
    # var
    ###############################################################################################
    var_dict = read_and_proc.read_some_azimuth_fields(fileloc=[glob.glob(originpath+str(expname)+'/azim_'+str(varname)+'_*')[0]],fieldname=[varname])
    #ctrl_t2,ctrl_t4 = nearest_index(ctrlvar_dict['W']['W'].time/24,0.5)-1,nearest_index(ctrlvar_dict['W']['W'].time/24,7)-1
    r500=nearest_index(var_dict[varname][varname].radius,500)
    ###############################################################################################
    # dtheta
    ###############################################################################################
    if expname=='ctl':
        varL = var_dict[varname][varname]
    elif (expname=='ncrf_36h') or (expname=='lwcrf'):
        ctrlvar_dict = read_and_proc.read_some_azimuth_fields(fileloc=[glob.glob(originpath+'ctl/azim_'+str(varname)+'_*')[0]],fieldname=[varname])
        varL = read_and_proc.add_ctrl_before_senstart(ctrlvar_dict[varname][varname],var_dict[varname][varname],'NCRF36','Yes')
        del ctrlvar_dict
        gc.collect()
    elif (expname=='ncrf_60h'):
        ctrlvar_dict = read_and_proc.read_some_azimuth_fields(fileloc=[glob.glob(originpath+'ctl/azim_'+str(varname)+'_*')[0]],fieldname=[varname])
        varL = read_and_proc.add_ctrl_before_senstart(ctrlvar_dict[varname][varname],var_dict[varname][varname],'NCRF60','Yes')
        del ctrlvar_dict
        gc.collect()
    ###############################################################################################
    # smooth
    ###############################################################################################
    if expname=='ctl':
        var_s = gaussian_filter(varL,sigma=[3,0,0,0])
        del varL,var_dict
    else:
        var_s = gaussian_filter(varL,sigma=[3,0,0,0])
        del varL,var_dict    
    gc.collect()
    var_s = np.nan_to_num(var_s)
    print("---Finish!---")
    return var_s

In [18]:
ctl_lw,ctl_lwc = preproc_onevar('RTHRATLW','/scratch/06040/tg853394/tc/output/redux/maria/','ctl'),preproc_onevar('RTHRATLWC','/scratch/06040/tg853394/tc/output/redux/maria/','ctl')
ncrf_36h_lw,ncrf_36h_lwc = preproc_onevar('RTHRATLW','/scratch/06040/tg853394/tc/output/redux/maria/','ncrf_36h'),preproc_onevar('RTHRATLWC','/scratch/06040/tg853394/tc/output/redux/maria/','ncrf_36h')
ncrf_60h_lw,ncrf_60h_lwc = preproc_onevar('RTHRATLW','/scratch/06040/tg853394/tc/output/redux/maria/','ncrf_60h'),preproc_onevar('RTHRATLWC','/scratch/06040/tg853394/tc/output/redux/maria/','ncrf_60h')
lwcrf_lw,lwcrf_lwc = preproc_onevar('RTHRATLW','/scratch/06040/tg853394/tc/output/redux/maria/','lwcrf'),preproc_onevar('RTHRATLWC','/scratch/06040/tg853394/tc/output/redux/maria/','lwcrf')

---Finish!---
---Finish!---
---Finish!---
---Finish!---
---Finish!---
---Finish!---
---Finish!---
---Finish!---


In [19]:
radtype = 'ir'
if radtype=='all':
    ctrl_rad = ctl_lw+ctl_sw
    ncrf36_rad = ncrf_36h_lw+ncrf_36h_sw
    lwcrf_rad = lwcrf_lw+lwcrf_sw
    ncrf60_rad = ncrf_60h_lw+ncrf_60h_sw
elif radtype=='ir':
    ctrl_rad = ctl_lw-ctl_lwc
    ncrf36_rad = ncrf_36h_lw-ncrf_36h_lwc
    lwcrf_rad = lwcrf_lw-lwcrf_lwc
    ncrf60_rad = ncrf_60h_lw-ncrf_60h_lwc   
del ctl_lw,ctl_sw,ncrf_36h_lw,ncrf_36h_sw,ncrf_60h_lw,ncrf_60h_sw,lwcrf_lw,lwcrf_sw
gc.collect()

NameError: name 'ctl_sw' is not defined

In [ ]:
resi=False
if resi is True:
    c_th,n36_th,n60_th,l_th=derive_var.output_dtheta('/scratch/08350/tg876493/ruppert2020/','/scratch/06040/tg853394/tc/output/redux/maria/','ctl'),\
    derive_var.output_dtheta('/scratch/08350/tg876493/ruppert2020/','/scratch/06040/tg853394/tc/output/redux/maria/','ncrf_36h'),\
    derive_var.output_dtheta('/scratch/08350/tg876493/ruppert2020/','/scratch/06040/tg853394/tc/output/redux/maria/','ncrf_60h'),\
    derive_var.output_dtheta('/scratch/08350/tg876493/ruppert2020/','/scratch/06040/tg853394/tc/output/redux/maria/','lwcrf')
    
    ctrl_resi = c_th-ctrl_rad[:-1]
    ncrf36_resi = n36_th-ncrf36_rad[:-1]
    ncrf60_resi = n60_th-ncrf60_rad[:-1]
    lwcrf_resi = l_th-lwcrf_rad[:-1]
    del c_th,n36_th,n60_th,l_th,ctrl_rad,ncrf36_rad,ncrf60_rad,lwcrf_rad
    gc.collect()
    allrad = np.concatenate(([y for y in [ctrl_resi[24:120],ncrf36_resi[36:120],ncrf60_resi[60:120],lwcrf_resi[36:120]]]))
    del ctrl_resi,ncrf36_resi,ncrf60_resi,lwcrf_resi
else:
    index_arrays = [np.ones((x.shape), dtype=int)*i for i, x in enumerate([ctrl_rad[:-1][24:120],ncrf36_rad[:-1][36:120],ncrf60_rad[:-1][60:120],lwcrf_rad[:-1][36:120]])]
    allrad = np.concatenate(([y for y in [ctrl_rad[:-1][24:120],ncrf36_rad[:-1][36:120],ncrf60_rad[:-1][60:120],lwcrf_rad[:-1][36:120]]]))
    del ctrl_rad,ncrf36_rad,ncrf60_rad,lwcrf_rad
    gc.collect()

In [ ]:
preproc_noensemble.save_onevar(var=[allrad],varname=['ir'],savepath='/scratch/08350/tg876493/ruppert2020/output/heat/ir/',\
                              originpath='/scratch/06040/tg853394/tc/output/redux/maria/')
#read_and_proc.save_to_pickle('/scratch/08350/tg876493/ruppert2020/output/index_array',index_arrays) 

##### Remove diurnal effect

In [ ]:
#from scipy.ndimage import uniform_filter1d
#uniform_filter1d((obj),size=24,axis=0)
##############################################################################################
# (a) Urad, Vtan
UVdict = {}
dictname = ['ctrlU','ctrlV','ncrf36U','ncrf36V','ncrf60U','ncrf60V']
for indx,obj in tqdm(enumerate([ctrlhUrad,ctrlhVtan,ncrf_36hUrad,ncrf_36hVtan,ncrf_60hUrad,ncrf_60hVtan])):
    UVdict[dictname[indx]] = uniform_filter1d(np.swapaxes(np.asarray(obj),0,1),size=24,axis=0)
del ctrlhUrad,ctrlhVtan,ncrf_36hUrad,ncrf_36hVtan,ncrf_60hUrad,ncrf_60hVtan,dictname
gc.collect()
##############################################################################################
# (b) W, QV
WQVdict = {}
dictname = ['ctrlW','ctrlQV','ncrf36W','ncrf36QV','ncrf60W','ncrf60QV']
for indx,obj in tqdm(enumerate([ctrlvar_dict['W']['W'],ctrlvar_dict['QVAPOR']['QVAPOR'],NCRF36_w,NCRF36_qv,NCRF60_w,NCRF60_qv])):
    WQVdict[dictname[indx]] = uniform_filter1d((obj),size=24,axis=0)
del NCRF36_w,NCRF36_qv,NCRF60_w,NCRF60_qv,ctrlvar_dict,ncrf36var_dict,ncrf60var_dict
gc.collect()

#### Velocity time differences

In [5]:
ctl_du,ctl_dv,ctl_dw = preproc_noensemble.preproc_dudvdw('/scratch/08350/tg876493/ruppert2020/',\
                                                         '/scratch/06040/tg853394/tc/output/redux/maria/','ctl')
ncrf_36h_du,ncrf_36h_dv,ncrf_36h_dw = preproc_noensemble.preproc_dudvdw('/scratch/08350/tg876493/ruppert2020/',\
                                                                        '/scratch/06040/tg853394/tc/output/redux/maria/','ncrf_36h')
ncrf_60h_du,ncrf_60h_dv,ncrf_60h_dw = preproc_noensemble.preproc_dudvdw('/scratch/08350/tg876493/ruppert2020/',\
                                                                        '/scratch/06040/tg853394/tc/output/redux/maria/','ncrf_60h')
lwcrf_du,lwcrf_dv,lwcrf_dw = preproc_noensemble.preproc_dudvdw('/scratch/08350/tg876493/ruppert2020/',\
                                                               '/scratch/06040/tg853394/tc/output/redux/maria/','lwcrf')

---Finish!---
---Finish!---
---Finish!---
---Finish!---


In [6]:
ctl_dict = {"du":ctl_du,"dv":ctl_dv,"dw":ctl_dw}
del ctl_du,ctl_dv,ctl_dw
gc.collect()
ncrf_36h_dict = {"du":ncrf_36h_du,"dv":ncrf_36h_dv,"dw":ncrf_36h_dw}
del ncrf_36h_du,ncrf_36h_dv,ncrf_36h_dw
gc.collect()
ncrf_60h_dict = {"du":ncrf_60h_du,"dv":ncrf_60h_dv,"dw":ncrf_60h_dw}
del ncrf_60h_du,ncrf_60h_dv,ncrf_60h_dw
gc.collect()
lwcrf_dict = {"du":lwcrf_du,"dv":lwcrf_dv,"dw":lwcrf_dw}
del lwcrf_du,lwcrf_dv,lwcrf_dw
gc.collect()

0

In [7]:
alldu = np.concatenate(([y for y in [ctl_dict['du'][24:120],ncrf_36h_dict['du'][36:120],ncrf_60h_dict['du'][60:120],lwcrf_dict['du'][36:120]]]))
alldv = np.concatenate(([y for y in [ctl_dict['dv'][24:120],ncrf_36h_dict['dv'][36:120],ncrf_60h_dict['dv'][60:120],lwcrf_dict['dv'][36:120]]]))
alldw = np.concatenate(([y for y in [ctl_dict['dw'][24:120],ncrf_36h_dict['dw'][36:120],ncrf_60h_dict['dw'][60:120],lwcrf_dict['dw'][36:120]]]))

In [8]:
import importlib
importlib.reload(preproc_noensemble)
preproc_noensemble.save_onevar(var=[alldu],varname=['du'],savepath='/scratch/08350/tg876493/ruppert2020/output/dudvdw/',\
                               originpath='/scratch/06040/tg853394/tc/output/redux/maria/')
preproc_noensemble.save_onevar(var=[alldv],varname=['dv'],savepath='/scratch/08350/tg876493/ruppert2020/output/dudvdw/',\
                               originpath='/scratch/06040/tg853394/tc/output/redux/maria/')
preproc_noensemble.save_onevar(var=[alldw],varname=['dw'],savepath='/scratch/08350/tg876493/ruppert2020/output/dudvdw/',\
                               originpath='/scratch/06040/tg853394/tc/output/redux/maria/')

1it [00:19, 19.12s/it]
1it [00:18, 18.78s/it]
1it [00:20, 20.02s/it]
